Installing HADOOP

In [16]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
!tar xf hadoop-3.3.2.tar.gz

import os
os.environ["HADOOP_CONF_DIR"] = "/content/hadoop-3.3.2/etc/hadoop"
!hadoop namenode -format
!hadoop --daemon start namenode
!hadoop --daemon start datanode
!jps
os.environ["PATH"] += os.pathsep + "$HADOOP_HOME/bin"

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
--2023-12-04 12:53:34--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz.1’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  15.2MB/s    in 51s     

2023-12-04 12:54:26 (11.9 MB/s) - ‘hadoop-3.3.2.tar.gz.1’ saved [638660563/638660563]

/bin/bash: hadoop: command not found
/bin/bash: hadoop: command not found
/bin/bash: hadoop: command not found
22503 Jps
14109 SparkSubmit


In [4]:
%pip install pyspark
%pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
namenode_hostname = spark._jsc.hadoopConfiguration().get("fs.defaultFS").split("//")[1].split(":")[0]

print("Namenode hostname:", namenode_hostname)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


23/12/04 12:32:38 WARN Utils: Your hostname, codespaces-fdeb22 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/12/04 12:32:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 12:32:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Namenode hostname: /


In [5]:
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
# Import necessary libraries
import pandas as pd
import docx
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from py4j.java_gateway import java_import

# Read Word document using python-docx
file_path = "test.docx"
doc = docx.Document(file_path)

# Extract text from the Word document
paragraphs = doc.paragraphs
words = []
for paragraph in paragraphs:
    words += paragraph.text.split()

# Initialize Spark
spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Import Hadoop FileSystem
java_import(spark._jvm, 'org.apache.hadoop.fs.Path')

# Get Hadoop FileSystem instance
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())

# Check if file exists in HDFS
hdfs_file_path = "/docfiles/words.txt"
if not fs.exists(spark._jvm.Path(hdfs_file_path)):
    # If file does not exist, store words in HDFS
    words_rdd = spark.sparkContext.parallelize(words)
    words_rdd.saveAsTextFile(hdfs_file_path)

# Read words from HDFS
hdfs_words = spark.sparkContext.textFile(hdfs_file_path)

# Perform word counting
word_counts = hdfs_words.flatMap(lambda line: line.split(" "))\
                        .filter(lambda word: isinstance(word, str))\
                        .map(lambda word: (word, 1))\
                        .reduceByKey(lambda a, b: a + b)

# Show the total word count
total_word_count = word_counts.collect()
print(total_word_count)

# Show the total word count
total_word_count = word_counts.collect()
print(total_word_count)

23/12/04 12:55:53 ERROR FileOutputCommitter: Mkdirs failed to create file:/docfiles/words.txt/_temporary/0
23/12/04 12:55:53 ERROR Executor: Exception in task 0.0 in stage 7.0 (TID 14)
java.io.IOException: Mkdirs failed to create file:/docfiles/words.txt/_temporary/0/_temporary/attempt_202312041255532289954406980335609_0037_m_000000_0 (exists=false, cwd=file:/workspaces/Machine_Learning_Workflows)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1081)
	at org.apache.hadoop.mapred.TextOutputFormat.getRecordWriter(TextOutputFormat.java:113)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.initWriter(SparkHadoopWriter.scala:238)
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:126)
	at org.apache.spark.inter

Py4JJavaError: An error occurred while calling o217.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:407)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScop